In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read data
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/fundar/indice-mercado-trabajo-ingresos/'

df_people_2021 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_21.txt', delimiter=';', low_memory=False)
df_people_2022 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_22.txt', delimiter=';', low_memory=False)

df_houses_2021 = pd.read_csv(pathdata + 'data_input/hogar_tot_urb_3T_2021.txt', delimiter=';', low_memory=False)
df_houses_2022 = pd.read_csv(pathdata + 'data_input/hogar_tot_urb_3T_2022.txt', delimiter=';', low_memory=False)

In [3]:
merged_df = pd.merge(df_people_2021[['CODUSU', 'COMPONENTE']], df_people_2022[['CODUSU', 'COMPONENTE']], on=['CODUSU', 'COMPONENTE'], how='outer', indicator=True)

# Duplicate rows and display the left-only rows (2021) and right-only rows (2022)
duplicated_rows = merged_df['_merge'].value_counts()['both']
left_only_rows = merged_df['_merge'].value_counts()['left_only']
right_only_rows = merged_df['_merge'].value_counts()['right_only']

# Select the right-only and right observation from both rows in merged_df
right_only_rows = merged_df[merged_df['_merge'] == 'right_only']
left_only_rows = merged_df[merged_df['_merge'] == 'left_only']
both_rows = merged_df[merged_df['_merge'] == 'both']
right_both_rows = pd.concat([both_rows, right_only_rows])

columns = ['PONDERA', 'PONDII', 'PONDIIO']
df_people = [df_people_2021, df_people_2022]

for col in columns:
    for df in df_people:
        df[f'rel_{col}'] = df[col] / df[col].sum()

df_merged_with_copies = pd.concat([df_people_2021, df_people_2022])

df_people_2021_no_dupl = pd.merge(left_only_rows, df_people_2021, on=["CODUSU", "COMPONENTE"], how='inner')
df_people_2022_no_dupl = pd.merge(right_both_rows, df_people_2022, on=["CODUSU", "COMPONENTE"], how='inner')

df_merged_without_copies = pd.concat([df_people_2021_no_dupl, df_people_2022_no_dupl])

new_columns = ['P21_new', 'P47T_new', 'TOT_P12_new']

df_merged_without_copies[new_columns] = pd.NaT

mask_2021 = df_merged_without_copies['ANO4'] == 2021

for col in ['P21', 'P47T', 'TOT_P12']:
    mask = mask_2021 & (df_merged_without_copies[col] != -9)
    df_merged_without_copies.loc[mask, f'{col}_new'] = df_merged_without_copies.loc[mask, col].div(0.546358573)

for col in ['P21', 'P47T', 'TOT_P12']:
    mask = mask_2021 & (df_merged_without_copies[col] == -9)
    df_merged_without_copies.loc[mask, f'{col}_new'] = df_merged_without_copies.loc[mask, col].copy()

mask_2022 = df_merged_without_copies['ANO4'] == 2022

for col in ['P21', 'P47T', 'TOT_P12']:
    df_merged_without_copies.loc[mask_2022, f'{col}_new'] = df_merged_without_copies.loc[mask_2022, col].copy()
    
for col in ['PONDERA', 'PONDII', 'PONDIIO']:
    temp_rel = df_merged_without_copies[f'rel_{col}'].div(df_merged_without_copies[f'rel_{col}'].sum())
    pob_prom = (df_people_2021[col].sum() + df_people_2022[col].sum())/2
    df_merged_without_copies[f'{col}_new'] = temp_rel * pob_prom


In [11]:
df_merged_without_copies.to_csv(pathdata + 'data_output/Base_pool_solo_con_replicas_actuales.csv')

In [91]:
prueba = df_merged_without_copies[['CODUSU','P47T', 'P47T_new', 'P21', 'P21_new']]

In [33]:
df_merged_without_copies['CAT_INAC'].value_counts()

0    57934
3    30241
1    19811
5     9012
4     8377
7     2069
6      342
2      172
Name: CAT_INAC, dtype: int64

In [39]:
len(df_merged_without_copies)

127958

In [69]:
df_merged_without_copies[(df_merged_without_copies['PP03D']==0 & (df_merged_without_copies['PP03C']==0))]['PONDERA_new'].sum()

16267106.25754121

In [47]:
df_merged_without_copies[(df_merged_without_copies['ESTADO']!=1 & (df_merged_without_copies['PP03C']==0))].count()

CODUSU         126266
COMPONENTE     126266
_merge         126266
TRIMESTRE      126266
ANO4           126266
                ...  
P47T_new       126073
TOT_P12_new    126266
PONDERA_new    126266
PONDII_new     126266
PONDIIO_new    126266
Length: 162, dtype: int64

In [80]:
df_merged_without_copies['PP03C'].value_counts()

0    75386
1    47831
2     4741
Name: PP03C, dtype: int64

In [84]:
aver = df_merged_without_copies[(df_merged_without_copies['PP03C']==0)]['PP3E_TOT']

In [88]:
df_merged_without_copies['PP3E_TOT'].isna().value_counts()

True     73694
False    54264
Name: PP3E_TOT, dtype: int64

In [70]:
df_merged_without_copies[df_merged_without_copies['PP03C']==0]['PP10C'].value_counts().sort_index()

0.0    1693
1.0    1471
2.0    2005
Name: PP10C, dtype: int64

In [52]:
df_merged_without_copies[df_merged_without_copies['PP03D']==0]['ESTADO'].value_counts().sort_index()

1    49523
Name: ESTADO, dtype: int64

In [67]:
aver = df_merged_without_copies[(df_merged_without_copies['PP3E_TOT'] == df_merged_without_copies['PP3F_TOT'].str.replace(',', '.').astype(float))][['PP03C', 'PP03D','ESTADO']]


In [45]:
df_merged_without_copies[df_merged_without_copies['PP03D']==0].count()

CODUSU         49523
COMPONENTE     49523
_merge         49523
TRIMESTRE      49523
ANO4           49523
               ...  
P47T_new       49523
TOT_P12_new    49523
PONDERA_new    49523
PONDII_new     49523
PONDIIO_new    49523
Length: 162, dtype: int64

In [62]:
df_merged_without_copies['CAT_INAC'].value_counts()

127958

In [63]:
df_merged_without_copies['CAT_OCUP'].value_counts().sort_index()

57741

In [59]:
df_merged_without_copies[df_merged_without_copies['CAT_INAC']==0].count()


CODUSU         57934
COMPONENTE     57934
_merge         57934
TRIMESTRE      57934
ANO4           57934
               ...  
P47T_new       57741
TOT_P12_new    57934
PONDERA_new    57934
PONDII_new     57934
PONDIIO_new    57934
Length: 162, dtype: int64

In [57]:
df_merged_without_copies[df_merged_without_copies['CAT_INAC']==0].count()



CODUSU         57934
COMPONENTE     57934
_merge         57934
TRIMESTRE      57934
ANO4           57934
               ...  
P47T_new       57741
TOT_P12_new    57934
PONDERA_new    57934
PONDII_new     57934
PONDIIO_new    57934
Length: 162, dtype: int64

In [31]:
df_merged_without_copies[df_merged_without_copies['CAT_INAC']==0]['PONDERA_new'].sum()



19548792.35849291

In [16]:
df_merged_without_copies['P47T_new'].isna().value_counts()

False    127765
True        193
Name: P47T_new, dtype: int64